<a id="1"></a><h2></h2>
<div style="background-color:rgba(0, 153, 0, 0.5);border-radius: 15px 50px;display:fill">
    <h1><center>1. Introduction</center></h1>
</div>

<div style="color:white;display:fill;border-radius:8px;
            background-color:#DAA520;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>1.1 | Goal</b></p>
</div>

Welcome to the May edition of the 2022 Tabular Playground Series! This month's challenge is a binary classification problem that includes a number of different feature interactions. This competition is an opportunity to explore various methods for identifying and exploiting these feature interactions. It wouldn't have been possible without the open-source contributions of other participants, in particular: <br>

1) @wti200 work on feature interactions <br>
2) @cabaxiom for idenfitying unique_characters feature <br>
3) GM @ambrosm for three public notebooks which are main inspiration for this notebook <br>
4) GM @sudalairajkumar for listing all the Feature Interactions <br>




# The overview of the buidling of metaclassfier:

<div class="alert alert-block alert-info"> 
    The architecture of a meta learning model involves two or more base models, often referred to as level-0 models, and a meta-model that combines the predictions of the base models, referred to as a level-1 model.
      Level-0 Models (Base-Models): Models fit on the training data and whose predictions are compiled.<br>
      Level-1 Model (Meta-Model): Model that learns how to best combine the predictions of the base models. <br> 
      It worth noting that when the predictions from Level-0 classifiers are used as the inputs for the Level-1 classifier,extreme care care must be taken as it can lead to overfitting. The below figure represents framework of the meta learning. The Leve-0 models need to do prediction on all each single fold, and learn from every four folds from the full data.<br> 
    
</div>


<br>
<img src= "https://yanpuli.github.io/images/stacking.jpg" alt ="TPA MAY" style='width: 800px;'>

<a id="1"></a><h2></h2>
## <font color="#blue">The structure of notebook.</font>
### <a href='#1'>1. Introduction </a><br>
### <a href='#2'>2. Load data and preprocessing</a><br>
### <a href='#3'>3. Feature Engineering</a><br>
### <a href='#4'>4. Neural network based prediction</a><br>
### <a href='#5'>5. LGBM based prediction</a><br>
### <a href='#6'>6. XGB based prediction</a><br>
### <a href='#7'>7. HistGradient based prediction</a><br>
### <a href='#8'>8. Catboost based prediction</a><br>
### <a href='#9'>9. Adaboost based prediction</a><br>
### <a href='#10'>10. Random Forest based prediction</a><br>
### <a href='#11'>11. Meta classifiers- preprocessing</a><br>
### <a href='#12'>12. Meta classifiers</a><br>
### <a href='#13'>13. References</a><br>

<a id="2"></a><h2></h2>
<div style="background-color:rgba(0, 153, 0, 0.5);border-radius: 15px 50px;display:fill">
    <h1><center>2. Loading the data </center></h1>
</div>
<div style="color:white;display:fill;border-radius:8px;
            background-color:#DAA520;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>2.1 | Train data</b></p>
</div>

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.style as style 
style.use('ggplot')
import warnings
warnings.filterwarnings('ignore')

import plotly.offline as py 
from plotly.offline import init_notebook_mode, iplot
py.init_notebook_mode(connected=True) # this code, allow us to work with offline plotly version
import plotly.graph_objs as go # it's like "plt" of matplot



cell_hover = {  # for row hover use <tr> instead of <td>
    'selector': 'td:hover',
    'props': [('background-color', '#ffffb3')]
}
index_names = {
    'selector': '.index_name',
    'props': 'font-style: italic; color: darkgrey; font-weight:normal;'
}
headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #000000; color: white;'
}
from IPython.display import HTML
pd.set_option('display.max_columns', None)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
from cycler import cycler
from IPython.display import display
import datetime
import scipy.stats
import math
import random

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.calibration import CalibrationDisplay
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping
from tensorflow.keras.layers import Dense, Input, InputLayer, Add
from tensorflow.keras.utils import plot_model

plt.rcParams['axes.facecolor'] = '#0057b8' # blue
plt.rcParams['axes.prop_cycle'] = cycler(color=['#ffd700'] +
                                         plt.rcParams['axes.prop_cycle'].by_key()['color'][1:])
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from colorama import Fore

from pandas_profiling import ProfileReport
import seaborn as sns
from sklearn import metrics
from scipy import stats
import math

from tqdm.notebook import tqdm
from copy import deepcopy
# Defining all our palette colours.
primary_blue = "#496595"
primary_blue2 = "#85a1c1"
primary_blue3 = "#3f4d63"
primary_grey = "#c6ccd8"
primary_black = "#202022"
primary_bgcolor = "#f4f0ea"

primary_green = px.colors.qualitative.Plotly[2]

plt.rcParams['axes.facecolor'] = primary_bgcolor

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-may-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-may-2022/test.csv')
s=train.head()
s.style.set_table_styles([cell_hover, index_names])

<a id="3"></a><h2></h2>
<div style="color:white;display:fill;border-radius:8px;
            background-color:#DAA520;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>2.2 | Feature Engineering</b></p>
</div>

This below model is modified version of public notebooks of @ambrosm. Consider upvoting the original work. <br>
Two additional features are introduced (Feature interaction between f_00, f_01 with f_26 ) <br>

In [ ]:
features = [f for f in test.columns if f != 'id' and f != 'f_27']
float_features = [f for f in features if test[f].dtype == float]
for df in [train, test]:
    # Extract the 10 letters of f_27 into individual features
    for i in range(10):
        df[f'ch{i}'] = df.f_27.str.get(i).apply(ord) - ord('A')
        
    # unique_characters feature is from https://www.kaggle.com/code/cabaxiom/tps-may-22-eda-lgbm-model
    df["unique_characters"] = df.f_27.apply(lambda s: len(set(s)))
    
    # Feature interactions: create three ternary features
    # Every ternary feature can have the values -1, 0 and +1
    df['i_02_21'] = (df.f_21 + df.f_02 > 5.2).astype(int) - (df.f_21 + df.f_02 < -5.3).astype(int)
    df['i_05_22'] = (df.f_22 + df.f_05 > 5.1).astype(int) - (df.f_22 + df.f_05 < -5.4).astype(int)
    df['i_00_26'] = (df.f_00 + df.f_26 > 5.1).astype(int) - (df.f_00 + df.f_26 < -5.4).astype(int)  #addtional feature
    df['i_01_26'] = (df.f_01 + df.f_26 > 5.1).astype(int) - (df.f_01 + df.f_26 < -5.4).astype(int)  #addtional feature
    i_00_01_26 = df.f_00 + df.f_01 + df.f_26
    df['i_00_01_26'] = (i_00_01_26 > 5.0).astype(int) - (i_00_01_26 < -5.0).astype(int)
    
features = [f for f in test.columns if f != 'id' and f != 'f_27']
float_features = [f for f in features if test[f].dtype == float]
int_features = [f for f in features if test[f].dtype == int and f.startswith('f')]
ch_features = [f for f in features if f.startswith('ch')]
# test[features].head(2)
corr = train.corr().abs()
mask = np.triu(np.ones_like(corr, dtype=np.bool))

fig, ax = plt.subplots(figsize=(90, 50), facecolor=primary_bgcolor)
# ax.text(-1.1, 0.16, 'Correlation between the Continuous Features', fontsize=10, fontweight='bold', fontfamily='serif')
ax.text(-1.1, 0.3, 'There is no features that pass more than 0.32 correlation within each other', fontsize=13, fontweight='light', fontfamily='serif')


# plot heatmap
res=sns.heatmap(corr, mask=mask, annot=True, fmt=".2f", cmap='coolwarm',annot_kws={"size": 25},
            cbar_kws={"shrink": .2}, vmin=0, vmax=1)
res.set_xticklabels(res.get_xmajorticklabels(), fontsize = 28)
res.set_yticklabels(res.get_ymajorticklabels(), fontsize = 28)
# yticks
plt.yticks(rotation=0)
plt.show()


<div class="alert alert-block alert-info"> 📌 
     Great news, as we can from the above figure that two new features that were introdcued in this notebook i_00_26 and i_01_26 are showing good correlation with Target. Lets use them</div>

In [ ]:
# It has been noticed that in addtion to various features extracted in f_27, by label encoding the F_27 the results are improved slightly.
# there are several unique lables are noticed in f_27 column of test data set. Hence both train and test data is merged for label encoding.
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
temp_df =(pd.concat([train.f_27, test.f_27],ignore_index=True)).to_frame() # dataframe co
temp_df['f_27']  = le.fit_transform(temp_df['f_27'])
test['f_27']  = le.transform(test['f_27'])
train['f_27'] = le.transform(train['f_27'])

<a id="4"></a><h2></h2>
<div style="background-color:rgba(0, 153, 0, 0.5);border-radius: 15px 50px;display:fill">
    <h1><center>3. Modeling </center></h1>
</div>

<br>
<div style="color:white;display:fill;border-radius:8px;
            background-color:#DAA520;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>3.1 | Neural network based prediction</b></p>
</div>
This below model is modified version of public notebooks of @ambrosm. Consider upvoting the original work. The main differnce is number of Epchos and features <br>

In [ ]:
def my_model():
    """Simple sequential neural network with four hidden layers.
    
    Returns a (not yet compiled) instance of tensorflow.keras.models.Model.
    """
    activation = 'swish'
    inputs = Input(shape=(len(features)))
    x = Dense(64, kernel_regularizer=tf.keras.regularizers.l2(40e-6),
              activation=activation,
             )(inputs)
    x = Dense(64, kernel_regularizer=tf.keras.regularizers.l2(40e-6),
              activation=activation,
             )(x)
    x = Dense(64, kernel_regularizer=tf.keras.regularizers.l2(40e-6),
              activation=activation,
             )(x)
    x = Dense(16, kernel_regularizer=tf.keras.regularizers.l2(40e-6),
              activation=activation,
             )(x)
    x = Dense(1, #kernel_regularizer=tf.keras.regularizers.l2(1e-6),
              activation='sigmoid',
             )(x)
    model = Model(inputs, x)
    return model

plot_model(my_model(), show_layer_names=False, show_shapes=True)

In [ ]:
EPOCHS = 500
EPOCHS_COSINEDECAY = 150
CYCLES = 1
VERBOSE = 0 # set to 0 for less output, or to 2 for more output
DIAGRAMS = True
USE_PLATEAU = False
BATCH_SIZE = 2048
ONLY_FIRST_FOLD = False

# see https://keras.io/getting_started/faq/#how-can-i-obtain-reproducible-results-using-keras-during-development
np.random.seed(1)
random.seed(1)
tf.random.set_seed(1)
features = [f for f in test.columns if f != 'id']
def fit_model(X_tr, y_tr, X_va=None, y_va=None, run=0):
    """Scale the data, fit a model, plot the training history and optionally validate the model
    
    Returns a trained instance of tensorflow.keras.models.Model.
    
    As a side effect, updates y_va_pred, history_list and score_list.
    """
    global y_va_pred
    start_time = datetime.datetime.now()
    
    scaler = StandardScaler()
    X_tr = scaler.fit_transform(X_tr)
    
    if X_va is not None:
        X_va = scaler.transform(X_va)
        validation_data = (X_va, y_va)
    else:
        validation_data = None

    # Define the learning rate schedule and EarlyStopping
    lr_start=0.01
    if USE_PLATEAU and X_va is not None: # use early stopping
        epochs = EPOCHS
        lr = ReduceLROnPlateau(monitor="val_loss", factor=0.7, 
                               patience=4, verbose=VERBOSE)
        es = EarlyStopping(monitor="val_loss",
                           patience=12, 
                           verbose=1,
                           mode="min", 
                           restore_best_weights=True)
        callbacks = [lr, es, tf.keras.callbacks.TerminateOnNaN()]

    else: # use cosine learning rate decay rather than early stopping
        epochs = EPOCHS_COSINEDECAY
        lr_end = 0.0002
        def cosine_decay(epoch):
            # w decays from 1 to 0 in every cycle
            # epoch == 0                  -> w = 1 (first epoch of cycle)
            # epoch == epochs_per_cycle-1 -> w = 0 (last epoch of cycle)
            epochs_per_cycle = epochs // CYCLES
            epoch_in_cycle = epoch % epochs_per_cycle
            if epochs_per_cycle > 1:
                w = (1 + math.cos(epoch_in_cycle / (epochs_per_cycle-1) * math.pi)) / 2
            else:
                w = 1
            return w * lr_start + (1 - w) * lr_end

        lr = LearningRateScheduler(cosine_decay, verbose=0)
        callbacks = [lr, tf.keras.callbacks.TerminateOnNaN()]
        
    # Construct and compile the model
    model = my_model()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_start),
                  metrics='AUC',
                  loss=tf.keras.losses.BinaryCrossentropy())

    # Train the model
    history = model.fit(X_tr, y_tr, 
                        validation_data=validation_data, 
                        epochs=epochs,
                        verbose=VERBOSE,
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        callbacks=callbacks)

    history_list.append(history.history)
    callbacks, es, lr, history = None, None, None, None
    
    if X_va is None:
        print(f"Training loss: {history_list[-1]['loss'][-1]:.4f}")
    else:
        lastloss = f"Training loss: {history_list[-1]['loss'][-1]:.4f} | Val loss: {history_list[-1]['val_loss'][-1]:.4f}"
        
        # Inference for validation
        y_va_pred = model.predict(X_va, batch_size=len(X_va), verbose=0)
        #oof_list[run][val_idx] = y_va_pred
        
        # Evaluation: Execution time, loss and AUC
        score = roc_auc_score(y_va, y_va_pred)
        print(f"Fold {run}.{fold} | {str(datetime.datetime.now() - start_time)[-12:-7]}"
              f" | {lastloss} | AUC: {score:.5f}")
        score_list.append(score)
    return model, scaler


print(f"{len(features)} features")
history_list = []
score_list = []
kf = KFold(n_splits=5)
for fold, (idx_tr, idx_va) in enumerate(kf.split(train)):
    X_tr = train.iloc[idx_tr][features]
    X_va = train.iloc[idx_va][features]
    y_tr = train.iloc[idx_tr].target
    y_va = train.iloc[idx_va].target
    
    fit_model(X_tr, y_tr, X_va, y_va)
    if ONLY_FIRST_FOLD: break # we only need the first fold

print(f"OOF AUC:                       {np.mean(score_list):.5f}")

In [ ]:
%%time
print(f"{len(features)} features")

X_tr = train[features]
y_tr = train.target

predtest_list_cnn = []
predtrain_list_cnn=[]
for seed in range(10):
    # see https://keras.io/getting_started/faq/#how-can-i-obtain-reproducible-results-using-keras-during-development
    print(f"{seed:2}")
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    model, scaler = fit_model(X_tr, y_tr, run=seed)
    predtest_list_cnn.append(model.predict(scaler.transform(test[features]),batch_size=BATCH_SIZE, verbose=VERBOSE))
    predtrain_list_cnn.append(model.predict(scaler.transform(train[features]),batch_size=BATCH_SIZE, verbose=VERBOSE))  

<a id="5"></a><h2></h2>
<div style="color:white;display:fill;border-radius:8px;
            background-color:#DAA520;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>3.2 |  LGBM based prediction</b></p>
</div>

In [ ]:
from sklearn.model_selection import KFold
features = [f for f in test.columns if f != 'id']
ntrain = train[features].shape[0]
ntest = test.shape[0]
SEED = 0 # for reproducibility
NFOLDS = 5 # set folds for out-of-fold prediction
kf = KFold(n_splits= NFOLDS, random_state=SEED, shuffle=True)

# Class to extend the Sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)

In [ ]:
def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [ ]:
%%time
from lightgbm import LGBMClassifier

lgb_params = {
    'n_estimators': 300,
    'min_child_samples': 8,
    'max_bins': 51,
    
}

# Create Numpy arrays of train, test and target ( Survived) dataframes to feed into our models
y_train = train['target'].values
x_train = train[features].values # Creates an array of the train data
x_test = test[features].values # Creats an array of the test data
pred_test_list_clf= []
pred_train_list_clf = []
for seed in range(5):
    print(f"{seed:2}")
    lgb = SklearnHelper(clf=LGBMClassifier, seed=seed, params=lgb_params)
    lgb_oof_train, lgb_oof_test = get_oof(lgb,x_train, y_train, x_test) 
    score = roc_auc_score(y_train, np.array(lgb_oof_train))
    print(score)
    pred_train_list_clf.append(lgb_oof_train)
    pred_test_list_clf.append(lgb_oof_test)
lgb_oof_train=np.array(pred_train_list_clf).mean(axis=0)
lgb_oof_test=np.array(pred_test_list_clf).mean(axis=0)
score = roc_auc_score(y_train, np.array(lgb_oof_train))
print("final_LGBM_AUC",score)

<a id="6"></a><h2></h2>
<div style="color:white;display:fill;border-radius:8px;
            background-color:#DAA520;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>3.3 |  XGB based prediction</b></p>
</div>

In [ ]:
from xgboost import XGBClassifier
xgb_params = {
    'n_estimators': 100,
    'eval_metric': 'auc',
    'silent': 0,
    
}

# Create Numpy arrays of train, test and target ( Survived) dataframes to feed into our models
y_train = train['target'].values
x_train = train[features].values # Creates an array of the train data
x_test = test[features].values # Creats an array of the test data
pred_test_list_clf= []
pred_train_list_clf = []
for seed in range(2): # you can increase maximum value of range upto 10. I have choosen it to be 1 so that the runtime is shorter.
    print(f"{seed:2}")
    xgb = SklearnHelper(clf=XGBClassifier, seed=SEED, params=xgb_params)
    xgb_oof_train, xgb_oof_test = get_oof(xgb,x_train, y_train, x_test) 
    score = roc_auc_score(y_train, np.array(xgb_oof_train))
    print(score)
    pred_train_list_clf.append(xgb_oof_train)
    pred_test_list_clf.append(xgb_oof_test)
xgb_oof_train=np.array(pred_train_list_clf).mean(axis=0)
xgb_oof_test=np.array(pred_test_list_clf).mean(axis=0)
score = roc_auc_score(y_train, np.array(xgb_oof_train))
print("final_XGB_AUC",score)


<a id="7"></a><h2></h2>
<div style="color:white;display:fill;border-radius:8px;
            background-color:#DAA520;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>3.4 |  HistGradientBoostingClassifier</b></p>
</div>

In [ ]:
%%time
from sklearn.ensemble import HistGradientBoostingClassifier
features = [f for f in test.columns if f != 'id']
Hist_params = {
    'max_iter': 200,    
}

# Create Numpy arrays of train, test and target ( Survived) dataframes to feed into our models
y_train = train['target'].values
x_train = train[features].values # Creates an array of the train data
x_test = test[features].values # Creats an array of the test data
pred_test_list_clf= []
pred_train_list_clf = []
for seed in range(2): # you can increase maximum value of range upto 10. I have choosen it to be 1 so that the runtime is shorter.
    print(f"{seed:2}")
    Hist = SklearnHelper(clf=HistGradientBoostingClassifier, seed=SEED, params=Hist_params)
    Hist_oof_train, Hist_oof_test = get_oof(Hist,x_train, y_train, x_test) 
    score = roc_auc_score(y_train, np.array(Hist_oof_train))
    print(score)
    pred_train_list_clf.append(Hist_oof_train)
    pred_test_list_clf.append(Hist_oof_test)
Hist_oof_train=np.array(pred_train_list_clf).mean(axis=0)
Hist_oof_test=np.array(pred_test_list_clf).mean(axis=0)
score = roc_auc_score(y_train, np.array(Hist_oof_train))
print("final_HIST_AUC",score)

<a id="8"></a><h2></h2>
<div style="color:white;display:fill;border-radius:8px;
            background-color:#DAA520;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>3.5 |  CatBoostClassifier</b></p>
</div>

In [ ]:
%%time
from catboost import CatBoostClassifier
iterations = 4500
cat_params = {
    'iterations': iterations,
    'learning_rate': 0.02,
    'early_stopping_rounds': 150,
    'max_depth': 5,
    'eval_metric': 'Accuracy',
    'loss_function': 'Logloss',
    'verbose': int(iterations/10)    
}


# Create Numpy arrays of train, test and target ( Survived) dataframes to feed into our models
y_train = train['target'].values
x_train = train[features].values # Creates an array of the train data
x_test = test[features].values # Creats an array of the test data
pred_test_list_clf= []
pred_train_list_clf = []
for seed in range(2): # you can increase maximum value of range upto 10. I have choosen it to be 1 so that the runtime is shorter.
    print(f"{seed:2}")
    cat = SklearnHelper(clf=CatBoostClassifier, seed=SEED, params=cat_params)
    cat_oof_train, cat_oof_test = get_oof(cat,x_train, y_train, x_test) 
    score = roc_auc_score(y_train, np.array(cat_oof_train))
    print(score)
    pred_train_list_clf.append(cat_oof_train)
    pred_test_list_clf.append(cat_oof_test)
cat_oof_train=np.array(pred_train_list_clf).mean(axis=0)
cat_oof_test=np.array(pred_test_list_clf).mean(axis=0)
score = roc_auc_score(y_train, np.array(cat_oof_train))
print("final_AUC",score) 

<a id="9"></a><h2></h2>
<div style="color:white;display:fill;border-radius:8px;
            background-color:#DAA520;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>3.6 |  AdaBoostClassifier</b></p>
</div>

In [ ]:
%%time
from sklearn.ensemble import AdaBoostClassifier
iterations = 100
ada_params = {
    'n_estimators': iterations,  
}


# Create Numpy arrays of train, test and target ( Survived) dataframes to feed into our models
y_train = train['target'].values
x_train = train[features].values # Creates an array of the train data
x_test = test[features].values # Creats an array of the test data
pred_test_list_clf= []
pred_train_list_clf = []
for seed in range(2):   # you can increase maximum value of range upto 10. I have choosen it to be 1 so that the runtime is shorter.
    print(f"{seed:2}")
    ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED, params=ada_params)
    ada_oof_train, ada_oof_test = get_oof(ada,x_train, y_train, x_test) 
    score = roc_auc_score(y_train, np.array(ada_oof_train))
    print(score)
    pred_train_list_clf.append(ada_oof_train)
    pred_test_list_clf.append(ada_oof_test) 
ada_oof_train=np.array(pred_train_list_clf).mean(axis=0)
ada_oof_test=np.array(pred_test_list_clf).mean(axis=0)
score = roc_auc_score(y_train, np.array(ada_oof_train))
print("final_AUC",score)

<a id="10"></a><h2></h2>
<div style="color:white;display:fill;border-radius:8px;
            background-color:#DAA520;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>3.7 |  RandomForestClassifier</b></p>
</div>

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier
rf_params = {
    'n_estimators': 100,
}

# Create Numpy arrays of train, test and target ( Survived) dataframes to feed into our models
y_train = train['target'].values
x_train = train[features].values # Creates an array of the train data
x_test = test[features].values # Creats an array of the test data
pred_test_list_clf= []
pred_train_list_clf = []
for seed in range(2): # you can increase maximum value of range upto 10. I have choosen it to be 1 so that the runtime is shorter.
    print(f"{seed:2}")
    rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
    rf_oof_train, rf_oof_test = get_oof(rf,x_train, y_train, x_test) 
    score = roc_auc_score(y_train, np.array(rf_oof_train))
    print(score)
    pred_train_list_clf.append(rf_oof_train)
    pred_test_list_clf.append(rf_oof_test)
rf_oof_train=np.array(pred_train_list_clf).mean(axis=0)
rf_oof_test=np.array(pred_test_list_clf).mean(axis=0)
score = roc_auc_score(y_train, np.array(rf_oof_train))
print("final_AUC",score)



<a id="11"></a><h2></h2>
<div style="background-color:rgba(0, 153, 0, 0.5);border-radius: 15px 50px;display:fill">
    <h1><center>4. Meta Learning </center></h1>
</div>
Typically, a meta-model is trained on the predictions made by base models on out-of-sample data. That is, data not used to train the base models is fed to the base models, predictions are made, and these predictions, along with the expected outputs, provide the input and output pairs of the training dataset used to fit the meta-model. However, in this notebook I have used prediction of the whole data. <br>
<div class="alert alert-block alert-info">

Important Notes: 
1) Prediction from both CNN, XGB and other predictions are merged along with few other dominating features and then are used as training data for Meta learning classifier. <br>
2) To avoid overfitting, a meta-model should be fit with light-weight algorithums. In here, I used Linear regression. For experiment purpose I have tried several other algorithums also. 
.</div>
<div style="color:white;display:fill;border-radius:8px;
            background-color:#DAA520;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>4.1 |  Preprocessing for Meta learning</b></p>
</div>


In [ ]:
#merging all the Level-0 prediction of the traing data
Metainput_train2 =pd.DataFrame()
Metainput_train2['id']=train['id']
Metainput_train2['target']=train['target']
Metainput_train2['LGBM_train_pred'] = lgb_oof_train.ravel()
Metainput_train2['XGB_train_pred'] =  xgb_oof_train.ravel()
Metainput_train2['CNN_train_pred'] =  np.array(predtrain_list_cnn).mean(axis=0)
Metainput_train2['HIST_train_pred'] =  Hist_oof_train.ravel()
Metainput_train2['catboost'] =  cat_oof_train.ravel()
Metainput_train2['adaboost'] =  ada_oof_train.ravel()
Metainput_train2['rf'] =  rf_oof_train.ravel()
Metainput_train2.head(2)

In [ ]:
#merging all the Level-0 prediction of the test data
Metainput_test2 =pd.DataFrame()
Metainput_test2['id']=test['id']
Metainput_test2['LGBM_train_pred'] = lgb_oof_test.ravel()
Metainput_test2['XGB_train_pred'] =  xgb_oof_test.ravel()
Metainput_test2['CNN_train_pred'] =  np.array(predtest_list_cnn).mean(axis=0)
Metainput_test2['HIST_train_pred'] =  Hist_oof_test.ravel()
Metainput_test2['catboost'] =  cat_oof_test.ravel()
Metainput_test2['adaboost'] =  ada_oof_test.ravel()
Metainput_test2['rf'] =  rf_oof_test.ravel()
Metainput_test2.head(2)


In [ ]:
imp_features_notarget = [x for x in Metainput_train2.columns if x != 'target']
from sklearn.model_selection import StratifiedKFold
val = np.zeros(Metainput_train2.shape[0])
pred = np.zeros(Metainput_train2.shape[0])
x = Metainput_train2[imp_features_notarget].values
y = Metainput_train2['target'].values

<a id="12"></a><h2></h2>
<div style="color:white;display:fill;border-radius:8px;
            background-color:#DAA520;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>4.1 |  Meta-learning classifier- LinearRegression</b></p>
</div>

it is extremly important that we do not over fit the Meta-learning classifier. For this reason very light weight classfier should be used. lets start with Linear regression first and see how it predicts.

In [ ]:
from sklearn.linear_model import LinearRegression
model_lr = LinearRegression()
pred_list_lr = []
for seed in range(10):
    print(f"{seed:2}")
    folds = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)
    for fold_index, (train_index,val_index) in enumerate(folds.split(x,y)):
        print('Batch {} started...'.format(fold_index))
        bst = model_lr.fit(x[train_index],y[train_index])
        val[val_index] = model_lr.predict(x[val_index])
        print('auc of this val set is {}'.format(roc_auc_score(y[val_index],val[val_index])))
    pred_list_lr.append(model_lr.predict(Metainput_test2))


### Based on the above its can be seen that Linear repression based Meta-classifier is unable to increase the accuracy  when compared with level-0 CNN model.
### I have tried various models as Meta-classifier(Catboost, XGB, RF and Ridge...) with an object not to overfit the model. The following has proven to be best one.

<a id="13"></a><h2></h2>
<div style="color:white;display:fill;border-radius:8px;
            background-color:#DAA520;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>4.2 |  Meta learning classifier- RandomForestRegressor</b></p>
</div>

In [ ]:
from sklearn.metrics import auc
clf = RandomForestClassifier(
    n_estimators=5,
)
def compute_roc_auc(index):
    y_predict = clf.predict_proba(x.iloc[index])[:,1]
    fpr, tpr, thresholds = roc_curve(y.iloc[index], y_predict)
    auc_score = auc(fpr, tpr)
    
    return fpr, tpr, auc_score
cv = StratifiedKFold(n_splits=5, random_state=123, shuffle=True)
results = pd.DataFrame(columns=['training_score', 'test_score'])
fprs, tprs, scores = [], [], []


x = Metainput_train2[imp_features_notarget]
y = Metainput_train2['target']
x_test = Metainput_test2[imp_features_notarget].values
pred_metarf_list = []
for (train_meta, test_meta), i in zip(cv.split(x, y), range(10)):
    clf.fit(x.iloc[train_meta], y.iloc[train_meta])
    _, _, auc_score_train = compute_roc_auc(train_meta)
    print(auc_score_train)
    fpr, tpr, auc_score = compute_roc_auc(test_meta)
    scores.append((auc_score_train, auc_score))
    fprs.append(fpr)
    tprs.append(tpr)
    clf.fit(x.iloc[train_meta], y.iloc[train_meta])
    pred_metarf_list.append( (clf.predict_proba(Metainput_test2[imp_features_notarget].values)[:,1]))

In [ ]:
submission = test[['id']].copy()
submission['target'] = np.array(pred_metarf_list).mean(axis=0)
submission.to_csv('submission.csv', index=False)
submission

<a id="14"></a><h2></h2>
<div style="background-color:rgba(0, 153, 0, 0.5);border-radius: 15px 50px;display:fill">
    <h1><center>4. References </center></h1>
</div>

https://www.kaggle.com/code/javigallego/tps-apr22-eda-fe-lstm-tutorial#1-|-Introduction <br>
https://www.kaggle.com/code/andreshg/tps-apr-data-visualization-and-highlights <br>
https://machinelearningmastery.com/histogram-based-gradient-boosting-ensembles/ <br>
https://www.kaggle.com/code/serigne/stacked-regressions-top-4-on-leaderboard <br>
https://yanpuli.github.io/posts/2018/01/blog-post-10/ <br>
https://www.kaggle.com/code/remekkinas/ensemble-learning-meta-classifier-for-stacking#10.-Final-prediction-(ensemble-learning-with-meta-classifier)